<div style="text-align: center; padding-top: 30px; padding-bottom: 10px;">

<h1 style="font-size: 2.8em; font-weight: 600; margin-bottom: 0.2em;">
Regional Neural Network Model
</h1>

<p style="font-size: 1.2em; color: gray; font-style: italic; margin-top: 0;">
This notebook visualises the main results from the paper.
</p>

</div>

## 1. User Input

In [5]:

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import os
from models.regional_model.model_functions.helper_functions.prepare_data import Prepare
from utils import create_pred_input
from models import MultivariateModelRegional as Model       
import yaml
os.environ['PYTHONHASHSEED'] = str(0)
regions = {'Asia': [142, "CHN"], 'Europe': [150, 'DEU'], 'Africa': [2, 'ZAF'], 'Americas': [19, 'USA'], 'Oceania': [9, 'AUS']}

#prepare the data
data=pd.read_excel('../data/MainData.xlsx')
growth, precip, temp = Prepare(data)
x_train = {0:temp, 1:precip}

#summary statics for standardisation
mean_temp=np.nanmean(data["TempPopWeight"])
std_temp=np.nanstd(data["TempPopWeight"])
mean_precip=np.nanmean(data["PrecipPopWeight"])
std_precip=np.nanstd(data["PrecipPopWeight"])

pred_input, T, P= create_pred_input(mc=False, mean_T=mean_temp, std_T=std_temp, mean_P=mean_precip, std_P=std_precip)

with open(f'../results/paper/weights/regional_model/config_raw.yaml', "r") as file:
    cfg = yaml.safe_load(file)
    
################################################## User Input ##################################################
n_models = 5 
region='Americas'  #specify region to plot
###############################################################################################################


## 2.Results

In [6]:
results = dict(np.load(f'../results/paper/weights/regional_model/results.npy', 
                       allow_pickle=True).item())

results={k: v for k,v in results.items() if v is not None}
top_models = sorted(results, key=lambda node: results[node][0])[0:n_models]


## 3. Prediction over Temperature-Precipitation grid

In [7]:

model_surfaces = []
for idx, node in enumerate(top_models[0:n_models], 1):
    factory = Model(node, cfg, x_train, growth)
    factory.Depth=len(node)
    model=factory.get_model()
    weight_file = f'../results/paper/weights/regional_model/parameters/{node}.weights.h5'
    model.load_params(weight_file)

    pred_flat = model.model_visual[region].predict([pred_input]).reshape(-1,)
    Growth = pred_flat.reshape(T.shape)

    opacity = 0.3
    surf = go.Surface(
        x=T, y=P/1000, z=Growth, #ensure that the surfaces are meassured in meters instead of milimeters
        colorscale='Cividis',
        opacity=0.85,
        showscale=False,
        name=f'Model {node}'
    )

    model_surfaces.append(surf)
  
#calculate the average surface
z=np.mean([surf.z for surf in model_surfaces], axis=0).reshape(T.shape)

mean_surface = go.Surface(
        x=T, y=P/1000, z=z,
        colorscale='Cividis',
        opacity=0.85,
        showscale=False,
        name='mean_surface'
    )


2026-02-10 10:39:01.188801: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


## 4. Data visualization

In [8]:
plot_data = [mean_surface] 
fig = go.Figure(data=plot_data)

##layout for webpage 
fig.update_layout(
        autosize=True,
       
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0,
        ),

            scene=dict(
                xaxis_title='Temperature (°C)',
                yaxis_title='Precipitation (m)',
                zaxis=dict(title=dict(text="Δ ln(GDP)"),range=[-0.3, 0.3]),
                camera=dict(eye=dict(x=1.738, y=-1.780, z=0.589))
                
            ),
            
            legend=dict(
                bgcolor='rgba(255,255,255,0.7)',
                bordercolor='black',
                borderwidth=1
            ),
            font=dict(
            size=10
        )
        )

fig.show()
